<a href="https://colab.research.google.com/github/ViktorBakken/SM-r/blob/main/models/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

import scipy
import sklearn.linear_model as skl_lm

#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('png')
from IPython.core.pylabtools import figsize
figsize(10, 6) # Width and hight
#plt.style.use('seaborn-white')

In [4]:
biopsy = pd.read_csv(os.path.abspath("../data/normalized_labeled_training_data.csv"))
model = skl_lm.LogisticRegression(solver='lbfgs')
X_train = biopsy[['hour_of_day', 'day_of_week', 'month', 'holiday','weekday','summertime','temp','dew','humidity','precip','snow','snowdepth','windspeed','cloudcover','visibility']]
Y_train = biopsy['increase_stock']

model.fit(X_train,Y_train)
print("Model Summary")
print(model)

NameError: name 'pd' is not defined